## Projeto 2-1: Estabilização de imagens


### Entrega: 25/Set 23:59 via GitHub.

### Rubrica do Projeto:

    I. Não entregou ou entregou apenas um rascunho.
    D. O programa faz apenas uma estabilização parcial com os programas da aula de hoje.
    C. Utiliza o fluxo otico de uma janela no centro da imagem.
    B. Utiliza meios para compensar as faixas pretas nos cantos da imagem com algum limite.
    A. Consegue realizar a compensação em rotação no eixo de profundidade.
    
    +1/2 Conceito para implementações que comprovadamente melhoram o desempenho da estabilização.
    -1/2 Conceito se o notebook não contiver uma explicação detalhada da solução apresentada.

In [1]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt

Esse programa aplica estabilização para um vídeo gerado pela captura em tempo real da câmera do computador. Com esse propósito foi usado “good features to track” para gerar pontos da imagem do vídeo e  o método the Lucas-Kanade para seguir esses pontos frame a frame, sabendo a posição dos pontos em relação ao frame anterior é posição achar o vetor que descreve a velocidade em x e y de cada ponto, a média da velocidade de todos os pontos é considerada como a velocidade da imagem. Se a imagem for movida usando essa velocidade a gravação será estabilizada já que irá parecer que não teve movimento.

Outro aspecto importante é que a imagem mostrada só mostra uma parte de imagem filmada, isso porque quando há muito movimento em uma direção começam aparecer bordas pretas , já que temos somente a dimensão máxima da câmera, mostrando parte da imagem esse problema é escondido. O programa consegue também refazer as “good features to track” se houver menos que 10 pontos na imagem, isso garante que mesmo com movimentos bruscos haverá a reaquisição antes de todos os pontos sumirem, os pontos também podem ser readquiridos manualmente se a barra de espaço for pressionada.


In [5]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    if frame is None:
        break
        
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow.
    p1, st, err = cv2.calcOpticalFlowPyrLK(
        old_gray, frame_gray, p0, None, **lk_params)
    
    vecx = []
    vecy = []
    
    # if all points were lost terminate the program
    if p1 is not None:
        # if there are fewer than 10 points, get new points
        if (len(p1) < 10):
            p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
            p1, st, err = cv2.calcOpticalFlowPyrLK(
                old_gray, frame_gray, p0, None, **lk_params)
            movx =0
            movy =0
    else:
        break
    
    # calculates the change for each point
    for i,(point) in enumerate(p1):
        vec_p = (point[0]-(p0[i][0]))

        vecx.append((vec_p[0]))
        vecy.append((vec_p[1]))

        
    #calculates the average movemente
    movx += np.average(vecx)
    movy += np.average(vecy)

    #calculate were the img should move for stability
    M = np.array([[1,0,-movx],[0,1,-movy]])
    rows,cols,channel = frame.shape
    
        
    # Select good points.
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    
    # Draw the tracks.
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
    
    frame = cv2.add(frame, mask)
    #aplies the matriz M to the img
    img=cv2.warpAffine(frame,M,(cols,rows))
    
    #show only a small portion of the image
    img = img[100:400,150:450]
    
    
    
    cv2.imshow('frame', img)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    if k == 32: # if space is pressed get new points
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        p1, st, err = cv2.calcOpticalFlowPyrLK(
            old_gray, frame_gray, p0, None, **lk_params)
        movx =0
        movy =0
        
    # Now update the previous frame and previous points.
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()    
cv2.destroyAllWindows()